In [1]:
import powerlawrs 
import polars as pl
import numpy as np

In [2]:
file = "../reference_data/blackouts.txt"

# polars and pandas do NOT do a good job detecting headers, do not rely on them.
df = pl.read_csv(file, has_header=False)
data = df.to_series()

# Quick Start: Fit Pareto Type I

In [3]:
fit_results = powerlawrs.fit(data)

In [4]:
print(f"alpha: {fit_results.alpha}\nx_min: {fit_results.x_min}\nKS stat: {fit_results.D}\ntail length: {fit_results.len_tail}")

alpha: 1.2726372198302858
x_min: 230000.0
KS stat: 0.06067379629443781
tail length: 59


# API
## Stats module

In [5]:
powerlawrs.stats.descriptive.mean(data)

253868.68246445496

In [6]:
powerlawrs.stats.descriptive.variance(data, 1)

372476564023.59814

In [7]:
powerlawrs.stats.random.random_choice(data, 3)

[725000.0, 50000.0, 350000.0]

In [8]:
powerlawrs.stats.random.random_uniform(3)

[0.7795624730950734, 0.04262220617652024, 0.6383641530770068]

In [9]:
# Define a standard normal CDF in Python
import math
norm_cdf = lambda x: 0.5 * (1 + math.erf(x / math.sqrt(2.0)))

sorted_data = [-1.1, -0.5, 0.1, 0.2, 1.5]

# Call your Rust function, passing the Python function as an argument
(d_plus, d_minus, d_max) = powerlawrs.stats.ks.ks_1sam_sorted(sorted_data, norm_cdf)

print(f"D+: {d_plus}")
print(f"D-: {d_minus}")
print(f"D max: {d_max}")

D+: 0.22074029056089706
D-: 0.13982783727702897
D max: 0.22074029056089706


## Util module

In [10]:
powerlawrs.util.linspace(0,10,5)

[0.0, 2.5, 5.0, 7.5, 10.0]

In [11]:
simparams_dict = powerlawrs.util.sim.calculate_sim_params(0.01, data, 230000)
simparams_dict

{'num_sims_m': 2500,
 'sim_len_n': 211,
 'n_tail': 59,
 'p_tail': 0.2796208530805687}

In [12]:
# convert simparams dict to rust struct
simparams_struct = powerlawrs.util.sim.PySimParams(**simparams_dict)

# use the struct as an argument
sim_data = powerlawrs.util.sim.generate_synthetic_datasets(data, 230000, simparams_struct, 1.27)

In [13]:
#Note the library does not yet impliment zeta distribution for discrete data. 
pl.from_numpy(np.array(sim_data))

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,…,column_174,column_175,column_176,column_177,column_178,column_179,column_180,column_181,column_182,column_183,column_184,column_185,column_186,column_187,column_188,column_189,column_190,column_191,column_192,column_193,column_194,column_195,column_196,column_197,column_198,column_199,column_200,column_201,column_202,column_203,column_204,column_205,column_206,column_207,column_208,column_209,column_210
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
5300.0,50000.0,24506.0,20000.0,29000.0,261484.213675,81000.0,10000.0,319419.387821,511450.456049,88000.0,25000.0,254893.317661,4.8984e6,19000.0,375775.98182,173000.0,50000.0,164500.0,203000.0,10000.0,10000.0,1.9727e6,113200.0,75000.0,415897.367154,43696.0,40000.0,50000.0,163000.0,620527.834525,92000.0,100000.0,29900.0,25000.0,1.5041e6,18351.0,…,50000.0,50462.0,574809.505654,62000.0,164500.0,867629.253441,70000.0,20000.0,25000.0,60000.0,148000.0,18819.0,18000.0,55000.0,130000.0,12000.0,200000.0,80000.0,18819.0,113200.0,478216.724123,164500.0,38500.0,130000.0,29000.0,523517.222449,11529.0,114000.0,333762.312522,106850.0,173000.0,910789.497342,18351.0,33000.0,45000.0,45000.0,304230.566535
349529.921477,147000.0,797229.433085,335229.168362,71000.0,321393.491164,70000.0,29000.0,4.6250e6,50000.0,115000.0,81000.0,113200.0,29000.0,251569.260065,30500.0,50462.0,53000.0,122000.0,55000.0,112000.0,14273.0,18351.0,635913.310906,36073.0,231801.520681,70000.0,269402.633908,71000.0,1.1815e6,48000.0,25000.0,190000.0,287638.558218,241793.562165,38500.0,469648.886074,…,307006.266397,53000.0,12000.0,120000.0,65000.0,336678.418401,148000.0,323288.156637,60000.0,58000.0,190000.0,529809.240323,252804.618164,160000.0,71000.0,301905.825232,30001.0,133000.0,280639.712088,297393.56366,616119.734096,339573.398257,18000.0,36073.0,60000.0,92000.0,268289.815425,126000.0,504564.350311,71000.0,11529.0,164500.0,453539.704354,71000.0,25000.0,25000.0,48000.0
25000.0,12000.0,40911.0,320019.223212,133000.0,243814.861388,494856.419642,11000.0,278530.518886,10000.0,267779.84293,81000.0,70000.0,18819.0,53000.0,451716.721809,114000.0,164500.0,62000.0,43696.0,51000.0,38500.0,145000.0,3.4221e6,1.4776e6,11529.0,32000.0,65000.0,260464.813435,207200.0,25000.0,38500.0,60000.0,145000.0,455863.850351,45000.0,71000.0,…,75000.0,8000.0,1.3263e6,40000.0,17000.0,30001.0,70000.0,130000.0,1.0006e6,452376.332693,65000.0,80000.0,1.4773e6,15000.0,2900.0,408551.317877,523406.021074,48000.0,542281.80242,92000.0,18000.0,1.0673e6,114500.0,50462.0,191000.0,308595.533581,37000.0,70000.0,25000.0,50000.0,469033.62114,100000.0,114000.0,12000.0,38500.0,166000.0,122000.0
1800.0,25000.0,388718.935557,80000.0,40000.0,258537.318798,203000.0,18351.0,50000.0,15000.0,70000.0,1.2133e7,24000.0,71000.0,253729.8239,11529.0,494245.660752,10000.0,70000.0,50000.0,46000.0,279733.195096,25000.0,1.7232e6,386339.576103,55000.0,10000.0,100000.0,106850.0,285674.271787,29000.0,58000.0,35000.0,2900.0,20000.0,9000.0,100000.0,…,60000.0,164500.0,166000.0,50000.0,438030.027772,158000.0,37000.0,11529.0,133000.0,17000.0,164500.0,160000.0,75000.0,323943.124768,599569.8518,325203.481522,18000.0,24506.0,498841.080789,100000.0,388294.358036,11000.0,43000.0,12000.0,55000.0,26334.0,50462.0,70000.0,100000.0,15000.0,236459.905126,29000.0,24506.0,17000.0,48000.0,55000.0,50000.0
124000.0,2.6673e6,60000.0,30500.0,58000.0,50000.0,264849.221432,534496.635548,1000.0,50000.0,60000.0,324244.3

# Distributions
## Generic Power-Law

In [14]:
# instantiate the class
pl_class = powerlawrs.dist.powerlaw.Powerlaw(2.2726, 230000)

In [15]:
# pdf
pl_class.pdf(500000)

9.47430869971139e-07

In [16]:
# cdf
pl_class.cdf(500000)

0.627757791147596

In [17]:
# ccdf
pl_class.ccdf(500000)

0.372242208852404

In [18]:
# rv
# generate random U(0,1)
u = np.random.rand()
pl_class.rv(u)

829381.3643785068

## Pareto Type I

In [19]:
# instantiate the class
pareto_class = powerlawrs.dist.pareto.Pareto(1.2726, 230000)

In [20]:
# pdf
pareto_class.pdf(500000)

9.474308699711417e-07

In [21]:
# cdf
pareto_class.cdf(500000)

0.6277577911475959

In [22]:
# ccdf
pareto_class.ccdf(500000)

0.3722422088524041

In [23]:
# rv
# generate random U(0,1)
u = np.random.rand()
pareto_class.rv(u)

285822.4381280615

## Exponential

In [24]:
# instantiate the class
expo_class = powerlawrs.dist.exponential.Exponential(1.5)

In [25]:
# pdf
expo_class.pdf(2)

0.07468060255179593

In [26]:
# cdf
expo_class.cdf(2)

0.950212931632136

In [27]:
# ccdf
expo_class.ccdf(2)

0.04978706836786395

In [28]:
# rv
# generate random U(0,1)
u = np.random.rand()
expo_class.rv(u)

0.5237699718147185